# Grand Product Argument

Is and ARG commonly used today in zero-knowledge proving systems, like commimtment schemes, range proofs ..etc. 

The goal is for a prover $P$ to demostrate they posse a vector of n values $W$, $|W| = n$ which evaluate the a previous commitment $y$, s.t.: 
$y = \prod_{i=1}^{n} W_i$

For the purpose of this example, we limit the amounts of n to be a power of 2 $n = 2^a$. This way we can efficiently compute $y$ by generating binary tree, where the leafs values are equal to $V$, and each parent node is the multiplication of it's two children nodes. The resulting root node of the tree is equal to $y$.
$$\text{Binary Tree Example:}$$
$$\begin{array}{c}
     \text{Root} \\
     \begin{array}{cc}
         \text{L1} & \text{R1} \\
         \begin{array}{cc}
             \dots & \dots
         \end{array} &
         \begin{array}{cc}
             \dots & \dots
         \end{array}
     \end{array}
\end{array}$$


In [2]:
# number is a power of 2
def is_power_of_2(n: int):
    assert n & (n - 1) == 0

# return indexes of children nodes for a given node in the binary tree
def children(index: int, n: int) -> (int, int):
    assert n <= index <= 2 * n - 2
    left_child = (index - n) * 2
    return (left_child, left_child+1)

# create merkle tree
def create_tree(v: list[int]) -> list[int]:
    n = len(v)
    is_power_of_2(n)
    tree = v.copy()
    print(tree)
    for i in range(n, 2 * n - 1):
        left_i, right_i = children(i, n)
        result = tree[int(left_i)] * tree[int(right_i)]
        tree.append(result)

    assert len(tree) == 2 * n - 1
    return tree

We start by setting a couple variables up

In [3]:
a = 3
p = 120.next_prime()
F = GF(p)
P = PolynomialRing(F, 'x')
x = P.gen()

W = [F.random_element() for _ in range(2^a)]
n = len(W)

## Protocol

With the values for $V$ that we want to commit we can start the protocol

### 1. Commiting

We start by calculating $y$ and all it's intermidiate states. after that, $P$ commits to $y$, $W$, and  all it's in by generating the low degree of the Merkle Tree.

In [7]:
tree = create_tree(W)
y = tree[-1]
g = enumerate(tree)

lde = P.lagrange_polynomial(g)
lde

[118, 113, 69, 116, 58, 99, 124, 94]


44*x^14 + 103*x^13 + 80*x^12 + 89*x^11 + 80*x^10 + 126*x^9 + 67*x^8 + 51*x^7 + 94*x^6 + 68*x^5 + 101*x^4 + 24*x^3 + 93*x^2 + 118*x + 118

In [8]:
tree_indexes = list(range(n, 2*n -1))
left_children, right_children = zip(*[children(i, n) for i in tree_indexes])

left_child_poly = P.lagrange_polynomial(list(zip(tree_indexes, left_children)))
right_child_poly = P.lagrange_polynomial(list(zip(tree_indexes, right_children)))
assert left_child_poly + 1 == right_child_poly

h = lambda x: lde(x) - lde(left_child_poly(x)) * lde(right_child_poly(x))
assert all([h(i) == 0 for i in tree_indexes])

h_i = P.lagrange_polynomial(enumerate([h(i) for i in range(15)]))
print(f"h_i: {h_i}")

def vanishing_polynomial(points):
    z = 1
    for a in points:
        z *= (x - a)
    return z

Z = vanishing_polynomial(range(n))
print(f"vanishing polynomial: {Z}")

h_i: 42*x^14 + 119*x^13 + 94*x^12 + 117*x^11 + 81*x^10 + 73*x^9 + 104*x^8 + 92*x^7 + 50*x^6 + 64*x^5 + 108*x^4 + 114*x^3 + 99*x^2 + 26*x + 59
vanishing polynomial: x^8 + 99*x^7 + 68*x^6 + 72*x^5 + 38*x^4 + 76*x^3 + 114*x^2 + 40*x


In [12]:
r = F.random_element()

countient, remainder = h_i(r).quo_rem(Z(r))
assert remainder == 0